In [33]:
from softadapt import SoftAdapt, NormalizedSoftAdapt, LossWeightedSoftAdapt
import torch
# We redefine the loss components above for the sake of completeness.
# tensor([54241.1953, 54244.6250, 54214.0781, 54254.3125, 54214.7461, 54237.7031,
#         54219.9414, 54251.4375, 54226.9531, 54224.9141, 54236.0703, 54224.0938,
#         54238.4688, 54219.2773, 54247.1641, 54216.9844, 54250.0195, 54197.7344,
#         54223.5078, 54227.4805])
# tensor([0.1419, 0.1342, 0.1385, 0.1367, 0.1446, 0.1291, 0.1272, 0.1468, 0.1248,
#         0.1295, 0.1326, 0.1478, 0.1312, 0.1325, 0.1297, 0.1164, 0.1363, 0.1262,
#         0.1492, 0.1365])
loss_component_1 = torch.tensor([54370.5859, 54392.7109, 54375.5859, 54392.8438, 54420.3633, 54397.5234,
        54390.5078]).float()
loss_component_2 = torch.tensor([0.4527, 0.4667, 0.4634, 0.4577, 0.4569, 0.4411, 0.4395]).float()

print("Loss component 1: ", loss_component_1
      , "\nLoss component 2: ", loss_component_2)

#dtype
print("dtype of loss component 1: ", loss_component_1.dtype
      , "\ndtype of loss component 2: ", loss_component_2.dtype)

print(len(loss_component_1))
print(len(loss_component_2))

# Here we define the different SoftAdapt objects
softadapt_object  = SoftAdapt(beta=0.1)
normalized_softadapt_object  = NormalizedSoftAdapt(beta=0.1)
loss_weighted_softadapt_object  = LossWeightedSoftAdapt(beta=0.1)

Loss component 1:  tensor([54241.1953, 54244.6250, 54214.0781, 54254.3125, 54214.7461, 54237.7031,
        54219.9414]) 
Loss component 2:  tensor([0.1419, 0.1342, 0.1385, 0.1367, 0.1446, 0.1291, 0.1272])
dtype of loss component 1:  torch.float32 
dtype of loss component 2:  torch.float32
7
7


In [34]:
print(softadapt_object.get_component_weights(loss_component_1, loss_component_2, verbose=False))
# >>> tensor([2.8850e-01, 1.9070e-03, 1.1299e-22, 7.0959e-01], dtype=torch.float64)
print(normalized_softadapt_object.get_component_weights(loss_component_1, loss_component_2, verbose=False))
# >>> tensor([0.3221, 0.3251, 0.3528], dtype=torch.float64)

print(loss_weighted_softadapt_object.get_component_weights(loss_component_1, loss_component_2, verbose=False))
#>>> tensor([8.7978e-01, 1.2022e-01, 7.1234e-20]

tensor([1.0000e+00, 1.5598e-18], dtype=torch.float64)
tensor([0.5250, 0.4750], dtype=torch.float64)
Library
(tensor([54241.1953, 54244.6250, 54214.0781, 54254.3125, 54214.7461, 54237.7031,
        54219.9414]), tensor([0.1419, 0.1342, 0.1385, 0.1367, 0.1446, 0.1291, 0.1272]))
tensor([1.0000e+00, 3.9124e-24], dtype=torch.float64)
